In [1]:
import pandas as pd

In [ ]:
df=pd.read_csv("downloads/65k_enriched_topic.csv")

In [332]:
df["len"] = df.text.apply(lambda t: len(t.split(" ")))

In [333]:
#df = df[df["len"] > 7]

In [346]:
import math
def lookup_DOI(val, lookup, key="PMCID"): 
#     print(val, type(val))
#     if val is None or math.isnan(val):
#         return None
#     print(val)
    if type(val) == str:
        #print(val)
        #print(val, type(val))

        match = lookup[key] == val
        DOI = lookup['DOI'][match]
        return DOI.values[0]
    return None

df["pdoi"] = df['PMCID'].apply(lambda x: lookup_DOI(x, pcm, "PMCID"))

 ## TODOs:
 
 * find frequency of domain words within cluster
 * find subfrequency of that word (coocurrence)

# Keyphrase extraction

In [364]:
sentences = list(df["text"])
len(sentences)

56895

In [365]:
from sentence_transformers import SentenceTransformer, util
import time

print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/889 [00:00<?, ?it/s]

In [18]:
print("Start clustering")
start_time = time.time()

clusters = util.community_detection(s_embs, threshold=0.4)
print("Clustering took {:.2f} sec".format(time.time() - start_time), "len", len(clusters))

Start clustering


NameError: name 'embeddings' is not defined

In [2]:
!pip install keybert

In [2]:
df=pd.read_csv("downloads/57k_dois.csv")

,title,article_link,text,source,PMID,DOI,Create Date,coreId,oai,topic_name,topic_id,kws,kw1,kw2,domain_words,dd,d2
0,Dextran Fractional Clearance Studies in Acute ...,NaN,Dengue is a potentially serious common viral i...,core_pubmed,NaN,NaN,NaN,8608925.0,oai:pubmedcentral.nih.gov:3160290,dengue mosquito fever,22,"[('dengue potentially', 0.7544), ('dengue', 0....",dengue,viral infection,"['Dengue', 'treatment.']",NaN,NaN
1,[New insight into bacterial zoonotic pathogens...,NaN,The article also includes new emerging zoonoti...,a {negative_adjectives} ... {problem_nouns},25902698.0,10.13075/mp.5893.00084,NaN,NaN,NaN,streptococcus streptococci pharyngitis,91,"[('zoonotic bacteria', 0.6903), ('emerging zoo...",zoonotic bacteria,staphylococcus intermedius,"['The', 'zoonotic', ',', 'ECDC', ':', 'Strepto...",NaN,10.13075/mp.5893.00084
2,Protein Synthesis Attenuation by Phosphorylati...,NaN,Chagas' disease is a potentially life-threaten...,core_pubmed,NaN,NaN,NaN,8644680.0,oai:pubmedcentral.nih.gov:3218062,chagas cruzi trypanosoma,40,"[('chagas disease', 0.8), ('parasite trypanoso...",chagas disease,trypanosoma,"[""Chagas'"", 'life-threatening', 'unicellular',...",NaN,NaN
3,Mapping a gene for adult-onset primary open-an...,NaN,Glaucoma is the third-leading cause of blindne...,core_pubmed,NaN,NaN,NaN,7466653.0,oai:pubmedcentral.nih.gov:1712411,retinal glaucoma blindness,348,"[('cause blindness', 0.7102), ('glaucoma', 0.7...",glaucoma,blindness,"['Glaucoma', 'third-leading', 'world,', '>13.']",NaN,NaN
4,Role of immune cells in pancreatic cancer from...,NaN,Pancreatic cancer (PC) remains difficult to tr...,labels_oct7,27930550.0,10.1097/MD.0000000000005541,NaN,NaN,NaN,pancreatic adenocarcinoma pdac,34,"[('pancreatic cancer', 0.7705), ('pancreatic',...",pancreatic,anticancer therapies,"['Pancreatic', '(PC)', 'treat,', 'anticancer']",10.1097/MD.0000000000005541,10.1097/MD.0000000000005541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56880,Epidemiology of viral hepatitis B and C infect...,NaN,The global epidemic of hepatitis B and hepatit...,a {negative_adjectives} ... {problem_nouns},23008727.0,10.5812/hepatmon.6140,NaN,NaN,NaN,hcv hepatitis hdv,48,"[('hepatitis publichealth', 0.7003), ('epidemi...",hepatitis publichealth,hepatitis,"['The', 'B', 'C', 'publichealth', '.']",10.5812/hepatmon.6140,10.5812/hepatmon.6140
56881,Rodent model of reproductive tract leiomyomata...,NaN,Uterine myometrial tumors are the most commonl...,core_pubmed,NaN,NaN,NaN,7604771.0,oai:pubmedcentral.nih.gov:1870894,endometrial endometriosis gynecological,173,"[('myometrial tumors', 0.8078), ('gynecologica...",myometrial tumors,neoplasm women,"['Uterine', 'myometrial', 'gynecological', 'wo...",NaN,NaN
56882,Unexpected Death or Suicide by a Child or Adol...,NaN,The death of a young patient is a difficult bu...,core_pubmed,NaN,NaN,NaN,8661200.0,oai:pubmedcentral.nih.gov:3244296,dying terminally terminal,414,"[('young patient', 0.5394), ('death', 0.509), ...",death,death young,"['The', 'medicine.']",NaN,NaN
56883,RNAi for Plant Functional Genomics,NaN,A major challenge in the post-genome era of pl...,core_pubmed,NaN,NaN,NaN,8108330.0,oai:pubmedcentral.nih.gov:2447448,plant arabidopsis dormancy,163,"[('plant genome', 0.7875), ('genes plant', 0.6...",plant genome,plant biology,"['A', 'post-genome', 'genome.']",NaN,NaN


In [8]:
df["topic"] = df["topic_name"].apply(lambda terms: " ".join(terms.split(" ")[:2]))

In [11]:
df = df.drop(columns=["tn"])

In [13]:
df.to_csv ("downloads/octo_57k.csv", index = False, header=True)

In [377]:
#df.to_csv ("downloads/58k_enriched_topics.csv", index = False, header=True)

## Topic ANalysis

* check KW1 
* check sub KW2s for that KW

* group sentences (group their topics)
* build dictionaries of topics

In [20]:
from sentence_transformers import SentenceTransformer, util
import time

sentences = list(df["text"])
print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
s_embs = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/889 [00:00<?, ?it/s]

In [21]:
print("Start clustering")
start_time = time.time()

clusters = util.community_detection(s_embs, threshold=0.4)
print("Clustering took {:.2f} sec".format(time.time() - start_time), "len", len(clusters))

Start clustering
Clustering took 118.64 sec len 82


In [29]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [30]:
dump_tensors()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/util/__init__.py:23: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/torch/distributed/distributed_c10d.py:171: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  "torch.distributed.reduce_op is deprecated, please use "


Tensor: GPU pinned 56885 × 768
Tensor: GPU pinned 1 × 514
Parameter: GPU pinned 30527 × 768
Parameter: GPU pinned 514 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 32 × 12
Parameter: GPU pinned 768 × 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 3072 × 768
Parameter: GPU pinned 3072
Parameter: GPU pinned 768 × 3072
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 3072 × 768
Parameter: GPU pinned 3072
Parameter: GPU pinned 768 × 3072
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 3072 × 768
Parameter: GPU pinned 3072
Parameter: GPU pinned 768 × 3072
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU pinned 768
Parameter: GPU 

In [ ]:
print("Start clustering")
start_time = time.time()

clusters2 = util.community_detection(s_embs, threshold=0.2)
print("Clustering took {:.2f} sec".format(time.time() - start_time), "len", len(clusters))

Start clustering


In [22]:
#Print for all clusters the top 3 and bottom 3 elements
clus = {}
for i, cluster in enumerate(clusters[:20]):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    clus["cluster_number " + str(i)] = cluster
    
    for sentence_id in cluster[0:3]:
        print("\t", sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", sentences[sentence_id])


Cluster 1, #8317 Elements 
	 Dengue is a potentially serious common viral infection with no specific treatment.
	 The article also includes new emerging zoonotic bacteria , which are not currently monitored by ECDC but might pose a serious epidemiological problem in a foreseeable future : Streptococcus iniae , S. suis , S. dysgalactiae and staphylococci : Staphylococcus intermedius , S. pseudintermedius .
	 Chagas' disease is a potentially life-threatening illness caused by the unicellular protozoan parasite Trypanosoma cruzi.
	 ...
	 Xanthomonas oryzae pv oryzae (Xoo) causes bacterial blight, a serious disease of rice (Oryza sativa).
	 The transition from localized to systemic spreading of bacteria, viruses, and other agents is a fundamental problem that spans medicine, ecology, biology, and agriculture science.
	 The global epidemic of hepatitis B and hepatitis C is a serious publichealth problem .

Cluster 2, #2428 Elements 
	 The Silver-Russell syndrome (SRS) is characterised by s

In [ ]:
from sentence_transformers import SentenceTransformer, util
import time

words = list(set(df["kw1"]))
print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(words, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

In [41]:
from collections import Counter

def cluster_kw_frequencies(df, topic_name, col='kw1'):
    d = df[df["topic_name"] == topic_name]
    l = list(d[col])
    return Counter(l)

In [44]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [55]:
!pip install faiss-gpu

     |████████████████████████████████| 85.5 MB 30.8 MB/s            


In [56]:
import faiss

In [110]:
from sentence_transformers import SentenceTransformer, util
import time

words = list(set(list(df["kw2"]) + list(df["kw1"])))
print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(words, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/849 [00:00<?, ?it/s]

In [111]:
import numpy as np
emb = embeddings.cpu()
emb = emb.numpy()
d = emb.shape[1]
emb.shape, len(df)
#np.array(df["emb"][1])[:].shape
x = [list(e) for e in emb ]
#df["emb"] = x

In [112]:

def create_index(vectors):
    faiss_index = faiss.IndexFlatL2(len(vectors[0]))
    faiss_index.add(vectors)

#     print(faiss_index.ntotal)

    return faiss_index

#export
def query_index(text, embedder, target_list, index, with_distance=False, k=10):
    embedding = embedder.encode([text])
    distances, indices = index.search(embedding, k)
    if with_distance:
        return [(target_list[index], distances[0][i]) for i, index in enumerate(indices[0])]
    return [target_list[i] for i in indices[0]]

In [113]:
index = create_index(emb)

In [93]:
#kws = set(list(df["kw2"]))

In [106]:
stems = {}

for kw in [kw for kw in set(list(df["kw2"]) + list(df["kw1"])) if type(kw) == str]:
    stemmed = " ".join([Slice(k, 3).output for k in kw.split(" ")])
    if stems.get(stemmed):
        stems[stemmed] += [kw]
    else:
        stems[stemmed] = [kw]

In [122]:
" ".join("afe swe dcs".split(" ")[:2])

'afe swe'

In [127]:
df["tn"].unique()

array(['dengue mosquito', 'streptococcus streptococci', 'chagas cruzi',
       ..., 'histamine reserpine', 'mycoplasma arthritidis',
       'extraorally extrapyramidal'], dtype=object)

In [128]:
def cluster_kw_frequencies(df, topic_name, col='kw1'):
    d = df[df["tn"] == topic_name]
    l = list(d[col])
    return Counter(l)

cluster_kw_frequencies(df, "streptococcus streptococci")

Counter({'zoonotic bacteria': 1,
         'streptococcus': 5,
         'resistance streptococcus': 1,
         'pharyngitis gabhs': 1,
         'pathogen fish': 1,
         'streptococcal prevalence': 1,
         'rheumatic fever': 2,
         'streptococcus gas': 3,
         'phlegmonous gastritis': 1,
         'streptococci ggs': 1,
         'aggregatibacter': 1,
         'strep': 1,
         'lugdunensis endocarditis': 1,
         'streptococcus milleri': 1,
         'streptococcus equisimilis': 1,
         'staphylococcus lugdunensis': 1,
         'group streptococci': 1,
         'streptococci gbs': 1,
         'streptococci scrogroups': 1,
         'streptococcus gbs': 1})

In [124]:
query_index("cancer", model, words, index, True, 200)

[('cancer', 1.2424442e-12),
 ('cancers', 0.22477712),
 ('malignant cancer', 0.30477756),
 ('death cancer', 0.33443278),
 ('cancer disease', 0.33987525),
 ('cancer considered', 0.35273272),
 ('cancer tumor', 0.35458574),
 ('cancer highly', 0.39591017),
 ('malignant cancers', 0.41621506),
 ('cancers brc', 0.42654672),
 ('cancer death', 0.42936534),
 ('malignancy', 0.43267164),
 ('cancer worldwide', 0.43504712),
 ('cancer illnesses', 0.44005674),
 ('tumor', 0.44015333),
 ('cancer health', 0.44458184),
 ('malignance', 0.44556656),
 ('deadly cancer', 0.44936624),
 ('pain cancer', 0.45320347),
 ('nowadays cancer', 0.45395893),
 ('malignancies', 0.45613542),
 ('cancer nowadays', 0.4601174),
 ('carcinoma', 0.46399948),
 ('lung cancer', 0.4652792),
 ('tumors', 0.46972114),
 ('cancer continuously', 0.47097045),
 ('death cancers', 0.47581547),
 ('terminal cancer', 0.47896942),
 ('cancer life', 0.483144),
 ('cancer relatively', 0.48357186),
 ('war cancer', 0.48406583),
 ('cancer commonly', 0.48960

In [114]:
stems

{'space heating': ['space heating'],
 'heamodia treatment': ['heamodialysis treatment'],
 'thalasse constitu': ['thalassemia constitutes', 'thalassemia constitute'],
 'zinc': ['zinc'],
 'fragile infants': ['fragile infants'],
 'doping abuse': ['doping abuse'],
 'klebsiella species': ['klebsiella species'],
 'morning pills': ['morning pills'],
 'conside pharmaco': ['consider pharmacokinetic'],
 'absorptio grafted': ['absorption grafted'],
 'existe salmone': ['existence salmonella'],
 'perfora head': ['perforation head'],
 'edcs': ['edcs'],
 'nemali myopathy': ['nemaline myopathy'],
 'results hbv': ['results hbv'],
 'priority': ['priority'],
 'thorotra granula': ['thorotrast granulations'],
 'drug use': ['drug use'],
 'epide periodi': ['epidemics periodic'],
 'drawback flux': ['drawback flux'],
 'used journals': ['used journals'],
 'pathoge nairovi': ['pathogen nairovirus'],
 'adole public': ['adolescents public', 'adolescence public'],
 'syndrome srus': ['syndrome srus'],
 'permane fill

In [89]:
query_index("chaga disease", model, words, index, True, 50)

[('chagas disease', 0.29422185),
 ('causes chagas', 0.65663195),
 ('disease cgd', 0.75332314),
 ('syndrome cdgs', 0.88411117),
 ('cruzi disease', 0.89329267),
 ('disease african', 0.89801735),
 ('tropical disease', 0.90774506),
 ('disease cacao', 0.9088952),
 ('disease cjd', 0.91397667),
 ('disease cad', 0.9200283),
 ('disease igcd', 0.9300076),
 ('disease southwestern', 0.9422728),
 ('enigmatic disease', 0.9456077),
 ('barlow disease', 0.953141),
 ('discovery chagas', 0.95525414),
 ('tropical diseases', 0.9595732),
 ('maculopathy', 0.96156895),
 ('diseases tropical', 0.9628924),
 ('chlorosis', 0.9706787),
 ('skin disease', 0.9709861),
 ('disease considered', 0.9753209),
 ('fujimoto disease', 0.9793736),
 ('aaa disease', 0.9814501),
 ('chondromalacia', 0.98326993),
 ('body disease', 0.9843182),
 ('calciphylaxis', 0.9843546),
 ('dermatological disease', 0.98579615),
 ('human disease', 0.98724216),
 ('chiasmal syndrome', 0.9886662),
 ('dermatosis', 0.9903994),
 ('genetic disease', 0.9911

In [42]:
cluster_kw_frequencies(df, "chagas cruzi trypanosoma" )

Counter({'chagas disease': 3,
         'visceral leishmaniosis': 2,
         'cutaneous leishmaniasis': 8,
         'leishmaniasis vl': 14,
         'leishmaniasis': 22,
         'canine leishmaniasis': 2,
         'sleeping sickness': 2,
         'leishmania infantum': 1,
         'american trypanosomiasis': 1,
         'trypanosoma cruzi': 4,
         'trypanosomiasis hat': 3,
         'trypanosomiasis chagas': 2,
         'african trypanosomiasis': 10,
         'visceral leishmaniasis': 6,
         'trypanosoma': 2,
         'trypanosomatid mitochondria': 1,
         'canine leishmaniosis': 1,
         'leishmaniasis considered': 1,
         'carrier leishmaniasis': 1,
         'africa trypanosomiasis': 1,
         'infection leishmania': 1,
         'trypanosoniasis chagas': 1,
         'trypanosomes': 1,
         'leishmaniases': 2,
         'leishmania': 2,
         'leishmaniasis brazilian': 1,
         'trypanosomosis': 1,
         'africa trypanosomosis': 1,
         'leishman

In [66]:
class Slice(object):
    def __init__(self, root, leng):
        self.root = root # The original word
        self.leng = leng # How many morphemes should be used as the prefix for the new portmanteau
        self.morphemes = []
        self.output = '' #
        self.slice()

    def slice(self):
        import re
        ex = r'([^aeiou]*[aeiou]*)|[aeiou]*[^aeiou]*[aeiou]*'
        #ex = r'[^aeiou]*[aeiou]*[^aeiou]*|[aeiou]*[^aeiou]'
        root = self.root

        # Full list of morphemes for future use
        while root != '':
            end = re.match(ex, root).end()
            self.morphemes.append(root[0:end])
            root = root[end:]

        # Check that the number given isnt more than is available
        if len(self.morphemes) < self.leng:
            self.leng = len(self.morphemes)

        #Stitch together the output word
        g = 0
        while g < self.leng :
            self.output += self.morphemes[g]
            g += 1

In [212]:
for cluster_idxs in clusters[:1]:
    sample = df.iloc[cluster_idxs]
    sample

32541

In [214]:
#Print for all clusters the top 3 and bottom 3 elements
clus = {}
for i, cluster in enumerate(clusters[:20]):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    clus["cluster_number " + str(i)] = cluster
    
    for sentence_id in cluster[0:3]:
        print("\t", sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", sentences[sentence_id])


Cluster 1, #241 Elements 
	 Obesity represents a serious health problem as it is rapidly increasing worldwide .
	 Obesity is a serious and rapidly growing health problem worldwide .
	 Obesity is a serious public health problem that is growing alarmingly worldwide .
	 ...
	 Morbid obesity and its rapidly increasing prevalence has became a serious health , social and economic problem not only in Europe , but in other developed countries in the Western world .
	 Obesity is a serious health problem for all age groups and it also promotes Type 2 diabetes , cardiovascular diseases , cancer and hypertension .
	 Obesity is a serious health problem that increased risk for many complications , including diabetes and cardiovascular disease .

Cluster 2, #118 Elements 
	 Hepatitis C virus ( HCV ) is a serious global health problem and establishes chronic infection in a significant number of infected humans worldwide .
	 Hepatitis C virus ( HCV ) is a serious global health problem affecting approx

# NetworkX hierarchy

In [6]:
dd = df.copy()

In [17]:
dd["problem_id"] = [*range(0, len(dd))]

In [18]:
dd

,title,article_link,text,source,PMID,DOI,Create Date,coreId,oai,topic_name,topic_id,kws,kw1,kw2,domain_words,problem_id
0,Dextran Fractional Clearance Studies in Acute ...,NaN,Dengue is a potentially serious common viral i...,core_pubmed,NaN,NaN,NaN,8608925.0,oai:pubmedcentral.nih.gov:3160290,dengue mosquito fever,22,"[('dengue potentially', 0.7544), ('dengue', 0....",dengue,viral infection,"['Dengue', 'treatment.']",0
1,[New insight into bacterial zoonotic pathogens...,NaN,The article also includes new emerging zoonoti...,a {negative_adjectives} ... {problem_nouns},25902698.0,NaN,NaN,NaN,NaN,streptococcus streptococci pharyngitis,91,"[('zoonotic bacteria', 0.6903), ('emerging zoo...",zoonotic bacteria,staphylococcus intermedius,"['The', 'zoonotic', ',', 'ECDC', ':', 'Strepto...",1
2,Protein Synthesis Attenuation by Phosphorylati...,NaN,Chagas' disease is a potentially life-threaten...,core_pubmed,NaN,NaN,NaN,8644680.0,oai:pubmedcentral.nih.gov:3218062,chagas cruzi trypanosoma,40,"[('chagas disease', 0.8), ('parasite trypanoso...",chagas disease,trypanosoma,"[""Chagas'"", 'life-threatening', 'unicellular',...",2
3,Mapping a gene for adult-onset primary open-an...,NaN,Glaucoma is the third-leading cause of blindne...,core_pubmed,NaN,NaN,NaN,7466653.0,oai:pubmedcentral.nih.gov:1712411,retinal glaucoma blindness,348,"[('cause blindness', 0.7102), ('glaucoma', 0.7...",glaucoma,blindness,"['Glaucoma', 'third-leading', 'world,', '>13.']",3
4,Role of immune cells in pancreatic cancer from...,NaN,Pancreatic cancer (PC) remains difficult to tr...,labels_oct7,27930550.0,10.1097/MD.0000000000005541,NaN,NaN,NaN,pancreatic adenocarcinoma pdac,34,"[('pancreatic cancer', 0.7705), ('pancreatic',...",pancreatic,anticancer therapies,"['Pancreatic', '(PC)', 'treat,', 'anticancer']",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56890,Biological monitoring today and tomorrow.,NaN,The biggest problem with most biological monit...,limitation|problem ... with ... l=be that,7846491.0,NaN,NaN,NaN,NaN,congenital surgery autosomal,-1,"[('biological monitoring', 0.5138), ('biologic...",biological,problem biological,"['The', '.']",56890
56891,Rodent model of reproductive tract leiomyomata...,NaN,Uterine myometrial tumors are the most commonl...,core_pubmed,NaN,NaN,NaN,7604771.0,oai:pubmedcentral.nih.gov:1870894,endometrial endometriosis gynecological,173,"[('myometrial tumors', 0.8078), ('gynecologica...",myometrial tumors,neoplasm women,"['Uterine', 'myometrial', 'gynecological', 'wo...",56891
56892,Unexpected Death or Suicide by a Child or Adol...,NaN,The death of a young patient is a difficult bu...,core_pubmed,NaN,NaN,NaN,8661200.0,oai:pubmedcentral.nih.gov:3244296,dying terminally terminal,414,"[('young patient', 0.5394), ('death', 0.509), ...",death,death young,"['The', 'medicine.']",56892
56893,RNAi for Plant Functional Genomics,NaN,A major challenge in the post-genome era of pl...,core_pubmed,NaN,NaN,NaN,8108330.0,oai:pubmedcentral.nih.gov:2447448,plant arabidopsis dormancy,163,"[('plant genome', 0.7875), ('genes plant', 0.6...",plant genome,plant biology,"['A', 'post-genome', 'genome.']",56893


In [19]:
import networkx as nx

dd = dd.reset_index(drop=True)
samp = dd#.sample(frac=0.2)

def graph(df, meta_df):   
    G = nx.from_pandas_edgelist(df, 'kw2', 'kw1', create_using=nx.DiGraph()) #node_attr
    node_attr = meta_df.set_index("problem_id").to_dict('index')
    nx.set_node_attributes(G, node_attr)
    return G

In [24]:
G= graph(dd, dd)

In [25]:
nx.write_gexf(G, "downloads/" + ".gexf")

In [27]:
from networkx.algorithms.clique import find_cliques, find_cliques_recursive
from networkx.algorithms.cluster import generalized_degree, triangles
cl = find_cliques_recursive(G)
#gd = generalized_degree(G)

In [34]:
from networkx.algorithms import community

communities_generator = community.girvan_newman(G)

In [37]:
!pip install pyvis

In [39]:
from pyvis.network import Network
nt = Network('1000px', '1000px')
# populates the nodes and edges data structures
nt.from_nx(G)
nt.show('G.html')

AssertionError: 

In [3]:
import pandas as pd

df = pd.read_csv("downloads/57k_enriched_topics_kw.csv")

In [31]:
with open("vocab30k.txt") as f:
    vocab30k = f.read().split("\t\n")

def extract_domain_words(sent, vocab=vocab30k):
    words = sent.split(" ")
    domain = []
    for word in words:
        if word not in vocab:
            domain += [word]
    return domain

df["domain_words"] = df["text"].apply(extract_domain_words)

In [32]:
#df.to_csv ("downloads/57k_enriched_topics_kw.csv", index = False, header=True)

In [5]:
df[df["topic_name"] == "chagas cruzi trypanosoma"]

,title,article_link,text,source,PMID,DOI,Create Date,coreId,oai,topic_name,topic_id,kws,kw1,kw2,domain_words
2,Protein Synthesis Attenuation by Phosphorylati...,NaN,Chagas' disease is a potentially life-threaten...,core_pubmed,NaN,NaN,NaN,8644680.0,oai:pubmedcentral.nih.gov:3218062,chagas cruzi trypanosoma,40,"[('chagas disease', 0.8), ('parasite trypanoso...",chagas disease,trypanosoma,"[""Chagas'"", 'life-threatening', 'unicellular',..."
295,Mechanisms of resistance and susceptibility to...,NaN,Several animal models have been established to...,core_pubmed,NaN,NaN,NaN,8536862.0,oai:pubmedcentral.nih.gov:3052183,chagas cruzi trypanosoma,10,"[('visceral leishmaniosis', 0.6723), ('leishma...",visceral leishmaniosis,leishmaniosis,"['Several', 'visceral', 'leishmaniosis', '(VL)..."
613,Risk factors and prediction analysis of cutane...,NaN,Cutaneous leishmaniasis is currently a serious...,a {negative_adjectives} ... {problem_nouns},29578084.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('cutaneous leishmaniasis', 0.8051), ('leishm...",cutaneous leishmaniasis,leishmaniasis,"['Cutaneous', 'leishmaniasis', 'Africa', ',', ..."
640,New antigens for the serological diagnosis of ...,NaN,Visceral leishmaniasis ( VL ) still represents...,a {negative_adjectives} ... {problem_nouns},30571783.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('leishmaniasis vl', 0.7777), ('visceral leis...",leishmaniasis vl,leishmaniasis,"['Visceral', 'leishmaniasis', '(', 'VL', ')', ..."
893,The potential of live attenuated vaccines agai...,NaN,Cutaneous Leishmaniasis is a serious public he...,a {negative_adjectives} ... {problem_nouns},32027892.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('cutaneous leishmaniasis', 0.8306), ('leishm...",cutaneous leishmaniasis,leishmaniasis,"['Cutaneous', 'Leishmaniasis', ',', '.']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16810,Human African trypanosomiasis of the CNS: curr...,NaN,"Human African trypanosomiasis (HAT), also know...",core_pubmed,NaN,10.1172/JCI200421052,NaN,3408928.0,oai:pubmedcentral.nih.gov:338269,chagas cruzi trypanosoma,39,"[('trypanosomiasis hat', 0.7577), ('african tr...",trypanosomiasis hat,trypanosomiasis,"['Human', 'African', 'trypanosomiasis', '(HAT)..."
16971,A validated,NaN,Leishmaniasis is a serious health problem that...,a {negative_adjectives} ... {problem_nouns},33293176.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('leishmaniasis health', 0.7843), ('leishmani...",leishmaniasis,suitable vaccine,"['Leishmaniasis', '.']"
17087,Dispersion of Lutzomyia longipalpis and expans...,NaN,"In Brazil , visceral leishmaniasis ( VL ) is a...",a {negative_adjectives} ... {problem_nouns},30201037.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('leishmaniasis vl', 0.8235), ('visceral leis...",leishmaniasis vl,leishmaniasis,"['In', 'Brazil', ',', 'visceral', 'leishmanias..."
17094,"Use of crude, FML and rK39 antigens in ELISA t...",NaN,Visceral leishmaniasis is a disease caused by ...,a {negative_adjectives} ... {problem_nouns},21195550.0,NaN,NaN,NaN,NaN,chagas cruzi trypanosoma,10,"[('visceral leishmaniasis', 0.8356), ('leishma...",visceral leishmaniasis,leishmaniasis,"['Visceral', 'leishmaniasis', 'Leishmania', '(..."


In [3]:
from keybert import KeyBERT
kw_model = KeyBERT()

def get_kw(text, kw_model=kw_model):
    kws = kw_model.extract_keywords(text, 
                          keyphrase_ngram_range=(1, 2), 
                          stop_words='english',
                          use_mmr=True, 
                          diversity=0.2)
    
    kws.sort(key=lambda x: x[1], reverse=True)
    return kws

In [19]:
#df["kws"] = None

In [24]:
step=10000
for i in range(0, len(df), step):
    print(i)
    df.loc[i:i+step, "kws"] = df.loc[i:i+step, "text"].apply(get_kw)

0
10000
20000
30000
40000
50000


In [25]:
def likely(ts):
    #if a unigram is in the two highest likely probs THEN take the unigram]
    try:
        s0=ts[0][0]
        s1=ts[1][0]
        if len(s0.split(" ")) > 1 and len(s1.split(" ")) == 1:
            return s1
        return s0
    except:
        return None


In [26]:
def contender(topics):
    #take the first "different" option that breaks with the main options
    #iterate until the topic that breaks with the terms of previous topics
    terms = []
    idx=0
    for idx, t in enumerate(topics):
        words = t[0].split(" ")
#         print(words, "words")
        if len(set.intersection(set(words), set(terms))) < 1:
            break #word
        terms += words
    return likely(topics[idx+1:])

In [27]:
df["kw1"] = df.kws.apply(likely)

In [28]:
df["kw2"] = df.kws.apply(contender)